## sample code

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [2]:
# import torch

# with open("./temp.py") as f:
#     input_text = f.read()

# inputs = tokenizer(input_text, return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
# last_hidden_states.shape

## ~~~~

In [3]:
import os

fake_num = 50


path = "./code"
id_list = []
code_list = []

file_lst = os.listdir(path)
for file in file_lst:
    id_list.append(file[:-len(".py")])
    filepath = path + '/' + file
    with open(filepath) as f:
        code_list.append(f.read())

## 가짜 코드 생성
fake_id_list = []
fake_code_list = code_list * fake_num
for id in range(fake_num):
    fake_id_list.extend(map(lambda x: x+str(id), id_list))

In [4]:
batch_size = 64
max_len = 40
result = []

for idx in range(0, len(fake_id_list), batch_size):
    batch_input = tokenizer.batch_encode_plus(
        fake_code_list[idx:idx+batch_size],
        return_tensors="pt",
        max_length = max_len,
        truncation = True,
        pad_to_max_length = True,
        padding=True).to(device)
    batch_output = model(**batch_input)
    cls_embedding = batch_output.last_hidden_state[:, 0, :]
    result.append(cls_embedding.to('cpu'))
    del batch_input
    del batch_output
    del cls_embedding

    ## TODO 데이터 커지면 oom 뜸


In [5]:
total_result = torch.cat([r for r in result])
total_result.shape

torch.Size([250, 768])

In [6]:
from sklearn.cluster import MeanShift
import numpy as np

meanshift= MeanShift(bandwidth=1)
cluster_labels = meanshift.fit_predict(result_numpy)
print('cluster labels 유형:', np.unique(cluster_labels))

NameError: name 'result_numpy' is not defined